## Useful links
- vgg16 keras implementation - https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
- another vgg16 keras implementation - https://github.com/SamSamskies/courses/blob/master/deeplearning1/nbs/vgg16.py
- keras model API - https://keras.io/models/model/
- nvidia model implementation - https://github.com/jacobgil/keras-steering-angle-visualizations
- vehicle steering angle visualizations blog post - https://jacobgil.github.io/deeplearning/vehicle-steering-angle-visualizations
- comma ai - https://github.com/commaai/research/blob/master/train_steering_model.py

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D
import pandas as pd
import utils

IMAGE_SHAPE = (100, 320, 3)

def save_model(model, model_filename, weights_filename):
    if Path(model_filename).is_file():
        os.remove(model_filename)

    with open(model_filename,'w' ) as f:
        json.dump(model.to_json(), f)
        
    if Path(weights_filename).is_file():
        os.remove(weights_filename)
        
    model.save_weights(weights_filename)
    

def get_commaai_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1., input_shape=IMAGE_SHAPE))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")
              
    return model

Using TensorFlow backend.


In [2]:
model = get_commaai_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 100, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 25, 80, 16)    3088        lambda_1[0][0]                   
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 25, 80, 16)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 13, 40, 32)    12832       elu_1[0][0]                      
___________________________________________________________________________________________

In [3]:
# TODO: 
# 1. Make utils file with dataframe augmentation functions and load image function
# 2. Load a small sample subset and train model and try running the simulation with it
# 3. Try to figure out how to make a generator to use with fit_generator
df = utils.augment_dataframe(pd.read_csv('../data/' + 'driving_log.csv'))
df.hist(column='steering', bins=60)
df.head(20)

,image,is_flipped,steering
0,IMG/left_2016_12_01_13_31_13_279.jpg,False,0.25000
1,IMG/center_2016_12_01_13_31_13_279.jpg,False,0.00000
2,IMG/right_2016_12_01_13_31_13_279.jpg,False,-0.25000
3,IMG/left_2016_12_01_13_31_13_584.jpg,False,0.25000
4,IMG/center_2016_12_01_13_31_13_584.jpg,False,0.00000
5,IMG/right_2016_12_01_13_31_13_584.jpg,False,-0.25000
6,IMG/left_2016_12_01_13_31_14_092.jpg,False,0.25000
7,IMG/center_2016_12_01_13_31_14_092.jpg,False,0.00000
8,IMG/right_2016_12_01_13_31_14_092.jpg,False,-0.25000
9,IMG/left_2016_12_01_13_31_14_295.jpg,False,0.25000
